In [14]:
# def a function evaluate the fitting goodness of dataset by Gpr model and LOOCV
import math
import copy
import warnings
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import LeaveOneOut
warnings.filterwarnings('ignore')

def eval_dataset(dataset):
    dataset = np.array(dataset)
    KErnel = RBF(length_scale_bounds = (1e-2, 1e2))
    X = dataset[:, :-2]
    Y = dataset[:, -2:]
    loo = LeaveOneOut()
    y_pre_set = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, _ = Y[train_index], Y[test_index]
        Gpr_i = GaussianProcessRegressor(kernel=KErnel,
                                         alpha = 0.02,
                                        normalize_y=True,
                                        random_state=0).fit(X_train, y_train)

        y_pre = Gpr_i.predict(X_test, return_std=True)[0]
        y_pre_set.append(y_pre[0])
    
    Rvalue = PearsonR(y_pre_set, Y)
    return Rvalue


def PearsonR(X, Y,target=2):
    X_ = copy.deepcopy(np.array(X)).reshape(-1,2)
    Y_ = copy.deepcopy(np.array(Y)).reshape(-1,2)
    R_value = 0
    for j in range(target):
        X = X_[:,j]
        Y = Y_[:,j]
        xBar = np.mean(X)
        yBar = np.mean(Y)
        SSR = 0
        varX = 0
        varY = 0
        for i in range(0, len(X)):
            diffXXBar = X[i] - xBar
            diffYYBar = Y[i] - yBar
            SSR += (diffXXBar * diffYYBar)
            varX += diffXXBar ** 2
            varY += diffYYBar ** 2
        SST = math.sqrt(varX * varY)
    R_value += (1 - SSR / SST)
       
    return 1- R_value/target

In [15]:
eval_dataset(data)

0.8574465384014176

# LOF

In [16]:
import pandas as pd
import numpy as np
data = pd.read_csv('SACdataset.csv')

In [17]:
from sklearn.neighbors import LocalOutlierFactor
model = LocalOutlierFactor(n_neighbors=4, contamination=0.12,novelty=True)   
model.fit(data.iloc[:,:-2])

y = model._predict(data.iloc[:,:-2])
y

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1, -1, -1,  1,  1,  1,  1,  1,  1])

In [18]:
index = np.where(y == -1)[0]
index

array([18, 19, 20])

In [19]:
for datum in range(len(index)):
    print('Outliers LOF : \n')
    print(data.iloc[index[datum],:],'\n')

Outliers LOF : 

Sn        86.80
Bi         5.50
In         5.50
Ti         0.70
Ten      106.65
Elong     11.80
Name: 18, dtype: float64 

Outliers LOF : 

Sn        86.90
Bi         5.50
In         5.40
Ti         0.70
Ten      110.56
Elong     13.69
Name: 19, dtype: float64 

Outliers LOF : 

Sn        86.90
Bi         5.50
In         5.50
Ti         0.60
Ten      113.44
Elong     12.64
Name: 20, dtype: float64 



In [20]:
data_removed = data.drop(index,axis = 0)
data_removed

,Sn,Bi,In,Ti,Ten,Elong
0,92.9,3.0,2.5,0.1,64.60,20.85
1,92.7,3.0,2.5,0.3,62.59,23.69
2,92.5,3.0,2.5,0.5,72.05,22.84
3,91.9,3.0,3.5,0.1,66.60,19.79
4,91.7,3.0,3.5,0.3,70.29,21.60
5,91.5,3.0,3.5,0.5,68.33,15.95
6,91.9,4.0,2.5,0.1,77.65,16.44
7,91.7,4.0,2.5,0.3,80.49,16.64
8,91.5,4.0,2.5,0.5,81.03,15.66
9,90.9,4.0,3.5,0.1,78.59,18.44


In [21]:
print(eval_dataset(data))
print(eval_dataset(data_removed))

0.8574465384014176
0.48642230546348364


# iForest

In [22]:
from sklearn.ensemble import IsolationForest 
clf = IsolationForest( max_samples=100,random_state=0, contamination=0.05)
clf.fit(data.iloc[:,:-2])
y_pre = clf.predict(data.iloc[:,:-2])
y_pre

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1, -1,  1,  1,  1,  1,  1,  1,  1])

In [23]:
index_2 = np.where(y_pre == -1)[0]
index_2

array([18, 19])

In [24]:
for datum in range(len(index_2)):
    print('Outliers LOF : \n')
    print(data.iloc[index_2[datum],:],'\n')

Outliers LOF : 

Sn        86.80
Bi         5.50
In         5.50
Ti         0.70
Ten      106.65
Elong     11.80
Name: 18, dtype: float64 

Outliers LOF : 

Sn        86.90
Bi         5.50
In         5.40
Ti         0.70
Ten      110.56
Elong     13.69
Name: 19, dtype: float64 



In [25]:
data_removed_2 = data.drop(index_2,axis = 0)
data_removed_2

,Sn,Bi,In,Ti,Ten,Elong
0,92.9,3.0,2.5,0.1,64.60,20.85
1,92.7,3.0,2.5,0.3,62.59,23.69
2,92.5,3.0,2.5,0.5,72.05,22.84
3,91.9,3.0,3.5,0.1,66.60,19.79
4,91.7,3.0,3.5,0.3,70.29,21.60
5,91.5,3.0,3.5,0.5,68.33,15.95
6,91.9,4.0,2.5,0.1,77.65,16.44
7,91.7,4.0,2.5,0.3,80.49,16.64
8,91.5,4.0,2.5,0.5,81.03,15.66
9,90.9,4.0,3.5,0.1,78.59,18.44


In [26]:
print(eval_dataset(data))
print(eval_dataset(data_removed_2))

0.8574465384014176
0.5012970345580727


# TCGPR

In [27]:
from TCGPR import TCGPR
dataSet = "SACdataset.csv"
sampling_cap = 1
# for fdata screening (backward)
# ratio is recommend as a float <= 0.01
ratio = 0.03
up_search = 100

target=2

TCGPR.fit(filePath = dataSet,Sequence = 'backward',target=target,
          sampling_cap = sampling_cap, ratio = ratio,alpha = 0.02)

Modle Data screening of backward searching is executed : param initial_set_cap is masked 
The first execution of TCGPR : Data screening; backward version
It a greedy searching strategy, since the sampling_cap is 1
1-th iteration : The newly removed datum is
[92.4   1.5   4.5   0.1  74.64 18.81]
2-th iteration : The newly removed datum is
[91.5   3.    3.5   0.5  68.33 15.95]
****************************************************************************************************
The dataset after data screening is : 
      Sn   Bi   In   Ti    Ten  Elong
0  92.4  1.5  4.5  0.1  74.64  18.81
1  91.5  3.0  3.5  0.5  68.33  15.95
The changes of fitting goodness is : 
 [0.8564031494003133, 0.9183086261105824, 0.9591715716365259]
The dataset after data screening has been saved successfully!
****************************************************************************************************
One conherenced dataset has been saved !


In [1]:
from TCGPR import TCGPR

.. [TCGPR]: A Machine Learning Algorithms for Screening Abnormal Data and Important features of Small DataSet
.. [Info]: V1.2, Bin Cao (bcao@shu.edu.cn), MGI, SHU, Shanghai, CHINA.
References :
----------
.. [1] https://github.com/Bin-Cao/TCGPR/blob/main/Intro/TCGPR.pdf
.. [2] Software copyright 2022SR1423038 (2022): Zhang Tong-yi, Cao Bin, Sun Sheng.
.. [3] Patent CN 115017977 A(2022): Zhang Tong-yi, Cao Bin, Yuan Hao, Wei Qinghua, Dong Ziqiang.
